# Home work "CTC-loss"

Задача: реализовать прямой (forward) проход и вчислить матрицу $\alpha_t(s)$. Значения этой матрицы должны равняться
- $\alpha_t(s)$ в случае если валидные пути проходят через данную ячейку
- 0.0 в противном случае

Размерности входов и результата см. по коду.

Ноутбук с решением присыслать на voropaev@corp.mail.ru Тему письма пишите пожалуйста в формате "[МИФИ] Фамилия"

Deadline: 21.12.2018

In [1]:
import torch

In [2]:
# Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,2,3,1,4,5,4,3,2,1,], dtype=torch.long)

# y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1., 0., 0., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0,  0., 1., 0., 0., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 0., 0., 1.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],], dtype=torch.float32)

In [3]:
def labeled(l):
    l_ = [ ]
    for i in l:
        l_.append(0)
        l_.append(int(i))
    l_.append(0)
    return( l_ )

In [4]:
def compute_ctc_alpha(l, y):
    alpha = torch.zeros((2*l.size()[0]+1, y.size()[0]))
    l_ = labeled(l)
    
    #Initialisation
    for i in range(0,2):
        for j in range(0,y.shape[0]):
            if (i < (len(l_)-2*(y.shape[0] - j)-1)):
                alpha[i][j] = 0
            else:
                alpha[i][j] = y[j][l_[i]]
    
    #Forward
    for j in range(1,y.shape[0]):
        for i in range(2,len(l_)):
            if (i < (len(l_)-2*(y.shape[0] - j)-1)):
                alpha[i][j] = 0
            else:
                if (l_[i] != l_[i-2]):
                    alpha[i][j] = y[j][l_[i]]*(alpha[i-2][j-1] + alpha[i-1][j-1] + alpha[i][j-1])
                else:
                    alpha[i][j] = y[j][l_[i]]*(alpha[i-1][j-1] + alpha[i][j-1])
    
    
    return (alpha)

In [5]:
# Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha
def test():
    al = compute_ctc_alpha(l, y)
    ritght_al = torch.tensor([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    
    if torch.all(al == ritght_al):
        return True
    else:
        return False
    
assert test(), "Test faled"
    

In [6]:
#Второй вариант

In [7]:
def compute_ctc_alpha(l, y):
    alpha = torch.zeros((2*l.size()[0]+1, y.size()[0]))
    l_ = labeled(l)
    alpha[0][0] = y[0][l_[0]]
    alpha[1][0] = y[0][l_[1]]
    
    #Forward
    for j in range(1,y.shape[0]):
        for i in range(0,len(l_)):
            if (i < (len(l_)-2*(y.shape[0] - j)-1)):
                alpha[i][j] = 0
            else:
                if (l_[i] != l_[i-2]):
                    alpha[i][j] = y[j][l_[i]]*(alpha[i-2][j-1] + alpha[i-1][j-1] + alpha[i][j-1])
                else:
                    alpha[i][j] = y[j][l_[i]]*(alpha[i-1][j-1] + alpha[i][j-1])
    
    
    return (alpha)

In [8]:
# Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha
def test():
    al = compute_ctc_alpha(l, y)
    ritght_al = torch.tensor([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    
    if torch.all(al == ritght_al):
        return True
    else:
        return False
    
assert test(), "Test faled"
    